In [8]:
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image, ImageDraw
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch

processor = AutoImageProcessor.from_pretrained("nateraw/food")
model = AutoModelForImageClassification.from_pretrained("nateraw/food")

In [12]:
dataset = load_dataset("data_script/food101.py", trust_remote_code=True)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})